In [ ]:
import os
import pyttsx3
from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, clear_output
from tkinter import filedialog, Tk
import pandas as pd
from datetime import datetime
from rapidfuzz import fuzz
import ipywidgets as widgets
from transformers import pipeline
from googletrans import Translator

# Initialize translator
translator = Translator()

def translate_text(text, dest_language='hi'):
    try:
        translation = translator.translate(text, dest=dest_language)
        return translation.text
    except Exception as e:
        print(f"Translation error: {e}")
        return text

# Text-to-speech function
def speak(text):
    engine = pyttsx3.init()
    engine.say(text)
    engine.runAndWait()

# Analyze image filename for disease label
def analyze_image(img_path, threshold=75):
    filename = os.path.basename(img_path).lower()
    normalized_name = filename.replace('_', ' ').replace('-', ' ')

    disease_keywords = {
        "Brown leaf spot": ["spot", "leaf spot", "brown spot", "black spot", "bacterial spot"],
        "Nutrient deficiency": ["yellow", "chlorosis", "yellowing", "nutrient deficiency"],
        "Blight disease": ["blight", "early blight", "late blight"],
        "Powdery mildew": ["mildew", "powdery mildew", "powdery"],
        "Rust disease": ["rust", "rust disease"],
        "Wilt disease": ["wilt", "fusarium wilt", "verticillium wilt"],
        "Scab disease": ["scab"],
        "Mosaic virus infection": ["mosaic", "mosaic virus"],
        "Canker disease": ["canker"],
        "Dieback disease": ["dieback"],
        "Anthracnose fungal disease": ["anthracnose"],
        "Leaf curl disease": ["leaf curl", "leafcurl"],
        "Nematode infestation": ["nematode", "root knot nematode"],
        "Gall formation": ["gall", "root gall"],
        "Mold infection": ["mold", "gray mold", "botrytis"],
        "Root rot disease": ["root rot", "rootrot"],
        "Viral infection": ["virus", "viral infection"],
        "Black rot disease": ["black rot", "blackrot"],
        "Downy mildew": ["downy mildew"],
        "Healthy plant": ["healthy"]
    }

    for disease, keywords in disease_keywords.items():
        for kw in keywords:
            score = fuzz.partial_ratio(kw, normalized_name)
            if score >= threshold:
                return f"Detected: {disease}"

    return "Detected: Uncertain — recommend expert review"

# Watering advice based on weather, soil, and crop stage
def get_watering_advice(weather, soil, crop_stage="growing"):
    advice = ""

    if weather == "hot":
        if soil == "dry":
            advice = "Water immediately and deeply. Use mulch to conserve soil moisture."
        elif soil == "normal":
            advice = "Water early morning or late evening to reduce evaporation."
        else:
            advice = "Avoid watering; check for waterlogging and improve drainage."
    elif weather == "cool":
        if soil == "dry":
            advice = "Light watering recommended; avoid waterlogging."
        elif soil == "normal":
            advice = "Water moderately; maintain consistent moisture."
        else:
            advice = "Avoid watering; monitor drainage to prevent root diseases."
    else:  # rainy
        advice = "Usually no watering needed; monitor soil moisture and drainage."

    if crop_stage == "seedling":
        advice += " Seedlings need frequent but gentle watering to establish roots."
    elif crop_stage == "growing":
        advice += " Growing crops need regular watering to support growth."
    elif crop_stage == "mature":
        advice += " Mature plants may need reduced watering to prepare for harvest."
    elif crop_stage == "harvest":
        advice += " Minimize watering close to harvest to improve product quality."

    return f"Advice: {advice}"

# Solutions dictionary
def get_solution(diagnosis_label):
    suggestions = {
        "nutrient deficiency": "Use nitrogen-rich fertilizer like urea or compost. Conduct soil test for balanced nutrients.",
        "nitrogen deficiency": "Apply nitrogen fertilizers such as urea or ammonium nitrate.",
        "phosphorus deficiency": "Use phosphorus fertilizers like superphosphate or bone meal.",
        "potassium deficiency": "Apply potash fertilizers or potassium sulfate.",
        "pest attack": "Spray neem oil or organic insecticides. Monitor plants regularly and remove affected leaves.",
        "aphid infestation": "Introduce beneficial insects like ladybugs or use insecticidal soap.",
        "disease": "Apply copper-based fungicide and isolate the plant. Remove and destroy infected plant parts.",
        "bacterial spot": "Use copper sprays and avoid overhead irrigation.",
        "fungal infection": "Use antifungal sprays like neem oil, copper fungicides, or sulfur-based products. Ensure good airflow.",
        "powdery mildew": "Apply sulfur or potassium bicarbonate sprays. Remove affected leaves.",
        "watering issue": "Adjust watering schedule based on soil moisture. Avoid waterlogging or drought stress.",
        "overwatering": "Reduce watering frequency and improve soil drainage.",
        "underwatering": "Increase watering frequency, especially during hot and dry weather.",
        "root rot": "Improve drainage, avoid excess watering, and remove affected roots.",
        "viral infection": "Remove and destroy infected plants. Use resistant varieties.",
        "leaf curl disease": "Apply recommended insecticides to control vectors. Remove severely affected leaves.",
        "nematode infestation": "Practice crop rotation and use nematicides if necessary.",
        "healthy": "No action needed. Continue regular monitoring and good practices."
    }
    return suggestions.get(diagnosis_label.lower(), "No solution found. Please consult your local agricultural extension center.")

# Save results to CSV
def save_to_csv(farmer_name, description_text, nlp_label, cv_result, watering_tip, solution_text):
    data = {
        "Date": datetime.now().strftime("%Y-%m-%d %H:%M"),
        "Farmer Name": farmer_name,
        "Description": description_text,
        "Diagnosis (NLP)": nlp_label,
        "Diagnosis (CV)": cv_result,
        "Advice": watering_tip,
        "Solution": solution_text
    }
    df = pd.DataFrame([data])
    file_exists = os.path.isfile("plantifi_results.csv")
    if file_exists:
        df.to_csv("plantifi_results.csv", mode='a', header=False, index=False)
    else:
        df.to_csv("plantifi_results.csv", mode='w', header=True, index=False)
    print("✅ Saved to plantifi_results.csv")

# Initialize zero-shot classifier
labels = ["nutrient deficiency", "pest attack", "powdery mildew", "unknown issue"]
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# UI Widgets
farmer_name_input = widgets.Text(description='👤 Name:')
description_input = widgets.Textarea(description='Issue:', layout=widgets.Layout(width='100%', height='80px'))
weather_dropdown = widgets.Dropdown(options=['hot', 'cool', 'rainy'], value='hot', description='Weather:')
soil_dropdown = widgets.Dropdown(options=['dry', 'normal', 'wet'], value='dry', description='Soil:')
submit_button = widgets.Button(description="Run Diagnosis", button_style='success')
output = widgets.Output()

# Set your category_path and img_name here, or implement file upload for real use:
# For demo:
category_path = "PlantVillage/Healthy"  # Example category folder path
img_name = "healthy_leaf.jpg"           # Example image filename
img_path = os.path.join(category_path, img_name)

def on_submit_click(b):
    with output:
        clear_output()
        farmer_name = farmer_name_input.value.strip()
        description_text = description_input.value.strip()

        if not farmer_name or not description_text:
            print("Please fill in all required fields.")
            return

        # NLP classification
        nlp_result = classifier(description_text, labels)
        nlp_label = nlp_result['labels'][0] if nlp_result['labels'] else "Unknown"

        # CV diagnosis
        cv_result = analyze_image(img_path)

        weather = weather_dropdown.value
        soil = soil_dropdown.value

        watering_tip = get_watering_advice(weather, soil)
        solution_text = get_solution(nlp_label)

        # English response
        response = (
            f"Farmer: {farmer_name}\n"
            f"Issue: {description_text}\n"
            f"NLP Diagnosis: {nlp_label}\n"
            f"CV Diagnosis: {cv_result}\n"
            f"Watering Advice: {watering_tip}\n"
            f"Recommended Solution: {solution_text}"
        )

        print("----- English Output -----")
        print(response)

        # Translate to Hindi and display
        response_hi = translate_text(response, dest_language='hi')
        print("\n----- Hindi Translation -----")
        print(response_hi)

        # Speak Hindi translation
        speak(response_hi)

        save_to_csv(farmer_name, description_text, nlp_label, cv_result, watering_tip, solution_text)
        print("\n✅ Data saved successfully.")

submit_button.on_click(on_submit_click)

display(farmer_name_input, description_input, weather_dropdown, soil_dropdown, submit_button, output)


Device set to use cpu


Text(value='', description='👤 Name:')

Textarea(value='', description='Issue:', layout=Layout(height='80px', width='100%'))

Dropdown(description='Weather:', options=('hot', 'cool', 'rainy'), value='hot')

Dropdown(description='Soil:', options=('dry', 'normal', 'wet'), value='dry')

Button(button_style='success', description='Run Diagnosis', style=ButtonStyle())

Output()